In [1]:
from flask import Flask, jsonify, redirect, render_template
from scipy import stats
import numpy as np
import pandas as pd
import datetime as dt
from datetime import datetime
from dateutil.relativedelta import relativedelta
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [2]:
#################################################
# Database Setup
#################################################
engine = create_engine("sqlite:///project1_db.db")

# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
db_324 = Base.classes.db324
db_325 = Base.classes.db325
db_326 = Base.classes.db326
db_327 = Base.classes.db327
db_328 = Base.classes.db328
db_329 = Base.classes.db329
db_330 = Base.classes.db330
db_331 = Base.classes.db331
db_401 = Base.classes.db401
db_402 = Base.classes.db402
db_403 = Base.classes.db403
db_404 = Base.classes.db404
db_405 = Base.classes.db405
db_406 = Base.classes.db406
db_407 = Base.classes.db407
db_408 = Base.classes.db408
session = Session(engine)
#################################################
# Flask Setup
#################################################
app = Flask(__name__)


In [3]:
master_list_confirmed= []
master_list_confirmed.append(list(session.query(db_324.Confirmed).filter(db_324.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_325.Confirmed).filter(db_325.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_326.Confirmed).filter(db_326.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_327.Confirmed).filter(db_327.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_328.Confirmed).filter(db_328.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_329.Confirmed).filter(db_329.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_330.Confirmed).filter(db_330.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_331.Confirmed).filter(db_331.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_401.Confirmed).filter(db_401.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_402.Confirmed).filter(db_402.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_403.Confirmed).filter(db_403.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_404.Confirmed).filter(db_404.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_405.Confirmed).filter(db_405.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_406.Confirmed).filter(db_406.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_407.Confirmed).filter(db_407.Province_State == "Arizona")))
master_list_confirmed.append(list(session.query(db_408.Confirmed).filter(db_408.Province_State == "Arizona")))

# List of county names

In [4]:
county_list = list(session.query(db_403.Admin2).filter(db_403.Province_State == "Arizona"))
county_list1 = []
for i in county_list:
    county_list1.append(str(i[-1]))
county_list1

['Apache',
 'Cochise',
 'Coconino',
 'Gila',
 'Graham',
 'Greenlee',
 'La Paz',
 'Maricopa',
 'Mohave',
 'Navajo',
 'Pima',
 'Pinal',
 'Santa Cruz',
 'Unassigned',
 'Yavapai',
 'Yuma']

In [5]:
date_list = ["March_24","March_25","March_26","March_27","March_28","March_29","March_30","March_31","April_1",
             "April_2","April_3","April_4","April_5","April_6","April_7","April_8"]

# List of county names with lon/lat

In [6]:
county_list = list(session.query(db_403.Admin2).filter(db_403.Province_State == "Arizona"))
county_lon = list(session.query(db_403.Lat).filter(db_403.Province_State == "Arizona"))
county_lat = list(session.query(db_403.Long_).filter(db_403.Province_State == "Arizona"))
lon_lat = {county_list[i]:[str(county_lon[i]).strip("(,)"), str(county_lat[i]).strip("(,)")] for i in range(len(county_list))}
lon_lat_df = pd.DataFrame(lon_lat).drop(["Unassigned"], axis=1)
lon_lat_df

,Apache,Cochise,Coconino,Gila,Graham,Greenlee,La Paz,Maricopa,Mohave,Navajo,Pima,Pinal,Santa Cruz,Yavapai,Yuma
0,35.39465006,31.87934684,35.83883429,33.80190085,32.93166885,33.21498827,33.72854224,33.34835867,35.70471703,35.3997715,32.0971334,32.90525627,31.52508998,34.59933926,32.76895712
1,-109.4892383,-109.7516088,-111.7707178,-110.8132779,-109.8882178,-109.2405279,-113.9810029,-112.4918154,-113.7577902,-110.3218983,-111.7890033,-111.3449483,-110.8479088,-112.5538588,-113.90666740000002


# Dataframe of confirmed cases by county.

In [7]:
x=[str(l).strip("(,)") for i in master_list_confirmed for l in i]
n=16
final = [x[i * n:(i + 1) * n] for i in range((len(x) + n - 1) // n )]
dict = {}
count=0
for i in final:
    dict.update({date_list[count]:i})
    count+=1
county_confirmed_df = pd.DataFrame(dict,index = county_list1).drop(["Unassigned"])

In [8]:
json_confirmed_data_for_graphing1 = county_confirmed_df.to_json(orient='columns')
json_confirmed_data_for_graphing2 = county_confirmed_df.to_json(orient='index')
json_confirmed_data_for_graphing3 = county_confirmed_df.to_json(orient='values')

# Dataframe of confirmed cases by state.

In [9]:
az_confirmed_df = county_confirmed_df.astype('int64', copy=False)
az_confirmed_df.loc['Total'] = az_confirmed_df.sum()
az_confirmed_df = pd.DataFrame(az_confirmed_df.loc["Total"])
az_confirmed_df = az_confirmed_df.transpose()
az_confirmed_df

,March_24,March_25,March_26,March_27,March_28,March_29,March_30,March_31,April_1,April_2,April_3,April_4,April_5,April_6,April_7,April_8
Total,326,401,508,665,773,919,1157,1289,1530,1715,1937,2187,2486,2732,2870,3036


In [10]:
json_confirmed_data_for_graphing_state1 = az_confirmed_df.to_json(orient='columns')
json_confirmed_data_for_graphing_state2 = az_confirmed_df.to_json(orient='index')
json_confirmed_data_for_graphing_state3 = az_confirmed_df.to_json(orient='values')

In [11]:
master_list_deaths = []
master_list_deaths.append(list(session.query(db_324.Deaths).filter(db_324.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_325.Deaths).filter(db_325.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_326.Deaths).filter(db_326.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_327.Deaths).filter(db_327.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_328.Deaths).filter(db_328.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_329.Deaths).filter(db_329.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_330.Deaths).filter(db_330.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_331.Deaths).filter(db_331.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_401.Deaths).filter(db_401.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_402.Deaths).filter(db_402.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_403.Deaths).filter(db_403.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_404.Deaths).filter(db_404.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_405.Deaths).filter(db_405.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_406.Deaths).filter(db_406.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_407.Deaths).filter(db_407.Province_State == "Arizona")))
master_list_deaths.append(list(session.query(db_408.Deaths).filter(db_408.Province_State == "Arizona")))

# Dataframe of confirmed deaths by county.

In [12]:
x=[str(l).strip("(,)") for i in master_list_deaths for l in i]
n=16
final = [x[i * n:(i + 1) * n] for i in range((len(x) + n - 1) // n )]
dict = {}
count=0
new = []
for i in final:
    dict.update({date_list[count]:i})
    total=0
    count+=1
county_deaths_df = pd.DataFrame(dict,index = county_list1).drop(["Unassigned"])
county_deaths_df

,March_24,March_25,March_26,March_27,March_28,March_29,March_30,March_31,April_1,April_2,April_3,April_4,April_5,April_6,April_7,April_8
Apache,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cochise,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Coconino,0,1,1,2,2,2,2,2,2,5,5,10,10,10,10,14
Gila,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Graham,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Greenlee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
La Paz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Maricopa,2,2,2,5,5,5,5,5,5,12,12,28,31,31,35,37
Mohave,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Navajo,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1


In [13]:
json_deaths_data_for_graphing1 = county_deaths_df.to_json(orient='columns')
json_deaths_data_for_graphing2 = county_deaths_df.to_json(orient='index')
json_deaths_data_for_graphing3 = county_deaths_df.to_json(orient='values')

# Dataframe of deaths cases by state.

In [14]:
az_deaths_df = county_deaths_df.astype('int64', copy=False)
az_deaths_df.loc['Total'] = az_deaths_df.sum()
az_deaths_df = pd.DataFrame(az_deaths_df.loc["Total"])
az_deaths_df = az_deaths_df.transpose()
az_deaths_df

,March_24,March_25,March_26,March_27,March_28,March_29,March_30,March_31,April_1,April_2,April_3,April_4,April_5,April_6,April_7,April_8
Total,3,4,4,11,11,11,11,11,11,29,29,50,54,55,59,68


In [15]:
json_confirmed_data_for_graphing_state1 = az_deaths_df.to_json(orient='columns')
json_confirmed_data_for_graphing_state2 = az_deaths_df.to_json(orient='index')
json_confirmed_data_for_graphing_state3 = az_deaths_df.to_json(orient='values')